In [1]:
%pip install --upgrade --quiet  langchain langchain-community openai langchain-experimental neo4j tiktoken yfiles_jupyter_graphs python-dotenv json-repair langchain-openai langchain_core


Note: you may need to restart the kernel to use updated packages.


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
neo4j-graphrag 1.6.1 requires json-repair<0.40.0,>=0.39.1, but you have json-repair 0.41.1 which is incompatible.

[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
from langchain_core.runnables import  RunnablePassthrough, RunnableLambda
from langchain_core.prompts import ChatPromptTemplate
from pydantic import BaseModel, Field
from langchain_core.output_parsers import StrOutputParser
from langchain_community.graphs import Neo4jGraph
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_experimental.graph_transformers import LLMGraphTransformer
from neo4j import GraphDatabase
from yfiles_jupyter_graphs import GraphWidget
from langchain_community.vectorstores import Neo4jVector
from langchain_community.document_loaders import TextLoader
from langchain_community.vectorstores.neo4j_vector import remove_lucene_chars
import os
from neo4j import  Driver

from dotenv import load_dotenv

load_dotenv()

True

In [3]:
# Obtener variables de entorno
azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
azure_api_key = os.getenv("AZURE_OPENAI_KEY")
azure_deployment = os.getenv("GPT_ENGINE")
api_version = os.getenv("API_VERSION")

In [5]:
graph = Neo4jGraph(url="neo4j://localhost:7688", username="neo4j", password="your_password")

In [2]:
%pip install pypdf2 pytesseract pdf2image pillow


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
import os

# Usar una cadena cruda (raw string) para la ruta
os.environ['TESSDATA_PREFIX'] = r"C:\Program Files\Tesseract-OCR"

import pytesseract
from pdf2image import convert_from_path
from PIL import Image

# Asegúrate de poner tu ruta real si no agregaste Tesseract al PATH
pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'

# Ruta al PDF escaneado
pdf_path = './material/RTF 01640-4-2010 - Causalidad de gasto.pdf'

# Convertir el PDF a imágenes
pages = convert_from_path(pdf_path, dpi=300)

# Aplicar OCR
text = ""
for page in pages:
    text += pytesseract.image_to_string(page, lang='spa')

print(text[:1000])  # Mostramos los primeros caracteres para verificar


<>:2: SyntaxWarning: invalid escape sequence '\P'
<>:2: SyntaxWarning: invalid escape sequence '\P'
C:\Users\ExequielSilvestre\AppData\Local\Temp\ipykernel_39900\3851376376.py:2: SyntaxWarning: invalid escape sequence '\P'
  os.environ['TESSDATA_PREFIX'] = "C:\Program Files\Tesseract-OCR"
C:\Users\ExequielSilvestre\AppData\Local\Temp\ipykernel_39900\3851376376.py:2: SyntaxWarning: invalid escape sequence '\P'
  os.environ['TESSDATA_PREFIX'] = "C:\Program Files\Tesseract-OCR"


TesseractError: (1, 'Error opening data file C:\\Program Files\\Tesseract-OCR/spa.traineddata Please make sure the TESSDATA_PREFIX environment variable is set to your "tessdata" directory. Failed loading language \'spa\' Tesseract couldn\'t load any languages! Could not initialize tesseract.')

In [12]:
import os
import PyPDF2

reviews_folder = "./material"
documents = []

# Verifica si la carpeta existe
if os.path.isdir(reviews_folder):
    for filename in os.listdir(reviews_folder):
        file_path = os.path.join(reviews_folder, filename)

        # Verifica si el archivo es un PDF
        if filename.lower().endswith(".pdf"):
            try:
                with open(file_path, "rb") as f:
                    reader = PyPDF2.PdfReader(f)
                    text = "\n".join([page.extract_text() for page in reader.pages if page.extract_text()])
                    documents.append({"filename": filename, "content": text})
                    print(f"Text extracted from: {filename}")
            except Exception as e:
                print(f"Error reading {filename}: {e}")

else:
    print(f"The folder '{reviews_folder}' does not exist.")


Text extracted from: RTF 01640-4-2010 - Causalidad de gasto.pdf


In [14]:
for doc in documents:
    print(f"{doc['filename']} - Length of content: {len(doc['content'])}")

RTF 01640-4-2010 - Causalidad de gasto.pdf - Length of content: 0


In [13]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.schema import Document

# Configurar el splitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size=512, chunk_overlap=24)

# Convertir los documentos en chunks
chunked_documents = []
for doc in documents:
    chunks = text_splitter.split_text(doc["content"])
    for i, chunk in enumerate(chunks):
        chunked_documents.append(Document(page_content=chunk, metadata={"filename": doc["filename"], "chunk_id": i + 1}))

print(len(chunked_documents))
print(chunked_documents[:5])



0
[]


In [90]:
import os
from langchain.document_loaders import TextLoader
from langchain.chat_models import AzureChatOpenAI
from langchain.schema import SystemMessage


# # Definir los nodos y relaciones permitidos para la extracción de información
allowed_nodes = ["Patient", "Disease", "Medication", "Test", "Symptom", "Doctor"]
allowed_relationships = ["HAS_DISEASE", "TAKES_MEDICATION", "UNDERWENT_TEST", "HAS_SYMPTOM", "TREATED_BY"]


# Configurar Azure OpenAI
llm = AzureChatOpenAI(
    azure_endpoint=azure_endpoint,
    openai_api_key=azure_api_key,
    azure_deployment=azure_deployment,
    model=azure_deployment,  
    api_version=api_version,  
    temperature=0
)

# Configurar el transformador de gráficos con el prompt
llm_transformer = LLMGraphTransformer(
    llm=llm,
    allowed_nodes=allowed_nodes,
    allowed_relationships=allowed_relationships,
)
graph_documents = llm_transformer.convert_to_graph_documents(chunked_documents[:8])

In [91]:
graph_documents[0]

GraphDocument(nodes=[Node(id='Stomach Cancer', type='Disease', properties={}), Node(id='JOHN MULLEN, MD', type='Doctor', properties={}), Node(id='A 76-year-old woman', type='Patient', properties={}), Node(id='adenocarcinoma of the body of the stomach', type='Disease', properties={})], relationships=[Relationship(source=Node(id='A 76-year-old woman', type='Patient', properties={}), target=Node(id='Stomach Cancer', type='Disease', properties={}), type='HAS_DISEASE', properties={}), Relationship(source=Node(id='A 76-year-old woman', type='Patient', properties={}), target=Node(id='adenocarcinoma of the body of the stomach', type='Disease', properties={}), type='HAS_DISEASE', properties={}), Relationship(source=Node(id='JOHN MULLEN, MD', type='Doctor', properties={}), target=Node(id='A 76-year-old woman', type='Patient', properties={}), type='TREATED_BY', properties={})], source=Document(metadata={'filename': 'gastroesophageal-surgery-case-scenario-stomach-cancer-mullen.pdf', 'chunk_id': 1}

Para resetear el grafo en caso de ser necesario.

In [92]:
from neo4j import GraphDatabase

# Conectar a la base de datos Neo4j
driver = GraphDatabase.driver(uri="neo4j://localhost:7688", auth=("neo4j", "your_password"))

# Función para limpiar el grafo
def clear_graph(tx):
    tx.run("MATCH (n) DETACH DELETE n")

# Función para comprobar si el grafo está vacío (contar nodos y relaciones)
def check_empty_graph(tx):
    # Contar tanto los nodos como las relaciones
    result = tx.run("MATCH (n) OPTIONAL MATCH (n)-[r]->() RETURN count(n) AS node_count, count(r) AS relationship_count")
    for record in result:
        return record["node_count"], record["relationship_count"]

# Ejecutar la limpieza del grafo
def execute_clear_graph():
    with driver.session() as session:
        session.execute_write(clear_graph)
        print("Graph database cleared successfully.")

# Ejecutar la comprobación del estado del grafo
def execute_check_empty_graph():
    with driver.session() as session:
        node_count, relationship_count = session.execute_read(check_empty_graph)
        print(f"Node count: {node_count}, Relationship count: {relationship_count}")

# Limpiar el grafo y luego verificar
execute_clear_graph()          # Limpiar el grafo
execute_check_empty_graph()    # Verificar que está vacío


Graph database cleared successfully.
Node count: 0, Relationship count: 0


In [93]:
graph.add_graph_documents(
    graph_documents,
    baseEntityLabel=True,
    include_source=True
)

In [94]:
def showGraph():
    driver = GraphDatabase.driver(
        uri = "neo4j://localhost:7688",
        auth = (os.getenv("NEO4J_USERNAME"),
                os.getenv("NEO4J_PASSWORD"))
    )
    session = driver.session()
    widget = GraphWidget(graph=session.run("MATCH (s)-[r:!MENTIONS]->(t) RETURN s,r,t").graph())
    widget.node_label_mapping = 'id'
    return widget

showGraph()

GraphWidget(layout=Layout(height='800px', width='100%'))

In [109]:
import os
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores.neo4j_vector import Neo4jVector
from langchain_openai import AzureOpenAIEmbeddings

# Configurar Embeddings de Azure OpenAI
embeddings = AzureOpenAIEmbeddings(
    model="text-embedding-ada-002",
    openai_api_key=os.getenv("AZURE_OPENAI_KEY"), 
    azure_endpoint="https://rag-esilvestre.openai.azure.com", 
    deployment="text-embedding-ada-002", 
    openai_api_type="azure",
    openai_api_version="2023-07-01-preview"
)

# Crear índice vectorial en Neo4j
vector_index = Neo4jVector.from_existing_graph(
    embeddings,
    search_type="hybrid",
    node_label="Document",
    text_node_properties=["text"],
    embedding_node_property="embedding",
    url="neo4j://localhost:7688",
    username="neo4j",
    password="your_password",
)

# Convertir el índice en un Retriever
vector_retriever = vector_index.as_retriever()

In [110]:
driver = GraphDatabase.driver(
        uri = "neo4j://localhost:7688", 
        auth = (os.getenv("NEO4J_USERNAME"),
                os.getenv("NEO4J_PASSWORD"))
    )

def create_fulltext_index(tx):
    query = '''
    CREATE FULLTEXT INDEX `fulltext_entity_id` 
    FOR (n:__Entity__) 
    ON EACH [n.id];
    '''
    tx.run(query)

# Function to execute the query
def create_index():
    with driver.session() as session:
        session.execute_write(create_fulltext_index)
        print("Fulltext index created successfully.")

# Call the function to create the index
try:
    create_index()
except:
    pass

# Close the driver connection
driver.close()

In [111]:
from langchain.prompts import PromptTemplate
from langchain.chains import GraphCypherQAChain
from langchain.chat_models import AzureChatOpenAI
from langchain_core.prompts.prompt import PromptTemplate


# Configurar Azure OpenAI
llmGraph= AzureChatOpenAI(
    azure_endpoint=azure_endpoint,
    openai_api_key=azure_api_key,
    azure_deployment=azure_deployment,
    model=azure_deployment,
    api_version=api_version,
    temperature=0
)


# Usar GraphCypherQAChain para integrar las consultas al gráfico y la LLM
chain = GraphCypherQAChain.from_llm(
    llmGraph,
    graph=graph,
    verbose=True,
    allow_dangerous_requests=True,
    validate_cypher=True
)

# Función para hacer las preguntas all llm contra el graph
def ask_question(query):
    response = chain.invoke({"query": query})
    return response


Lo que ocurre por detras:
1. Conuslta el schema del grafo, de esta forma puede encontrar una query adecuada
2. Genera la consulta cypher.
3. Realiza una valdiaciòn de la conuslta cypher_query_corrector.
4. Ejecuta la consulta contra el grafo.
5. Una ultima consulta a un llm para obtener el reusltado final.

In [112]:
ask_question("List all the medications")



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (m:Medication) RETURN m
Full Context:
[{'m': {'id': 'Paxil'}}, {'m': {'id': 'lisinopril'}}, {'m': {'id': 'metformin'}}, {'m': {'id': 'Inderal'}}, {'m': {'id': 'omeprazole'}}, {'m': {'id': 'atorvastatin'}}]

> Finished chain.


{'query': 'List all the medications',
 'result': 'The medications listed are Paxil, lisinopril, metformin, Inderal, omeprazole, and atorvastatin.'}

In [100]:
def full_retriever(query: str):
    graph_data = ask_question(query)
    vector_data = [el.page_content for el in vector_retriever.invoke(query)]
    final_data = f"""Graph data:
{graph_data}
vector data:
{"#Document ".join(vector_data)}
    """
    print(final_data)
    return final_data

In [101]:
llm_final_answer = AzureChatOpenAI(
    azure_endpoint=azure_endpoint,
    openai_api_key=azure_api_key,
    azure_deployment=azure_deployment,
    model=azure_deployment,
    api_version=api_version,
    temperature= 0
)

final_prompt_structure = ChatPromptTemplate.from_messages(
    [
        ("system", "You are AI assistant and you will answers questions using graph data and the vector data."),
        ("human", """Answer the question based only on the following context:
{context}

Question: {query}
Use natural language and be concise.
Answer:"""),
    ]
)

# 3. Crea una función simple para procesar la entrada
def process_input(query):
    # Obtener el contexto usando el retriever
    context = full_retriever(query)
    
    # Preparar los datos para el prompt
    data = {
        "context": context,
        "query": query
    }

    final_prompt = final_prompt_structure.format(**data)
    response = llm_final_answer.invoke(final_prompt)
    return response.text()

In [114]:
query = "List the diseases"
response = process_input(query)
print("--------FINAL ANSWER--------")
print(response)



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (d:Disease) RETURN d
Full Context:
[{'d': {'id': 'moderately to poorly differentiated adenocarcinoma'}}, {'d': {'id': 'type 2 diabetes'}}, {'d': {'id': 'hypertension'}}, {'d': {'id': 'Stomach Cancer'}}, {'d': {'id': 'adenocarcinoma of the body of the stomach'}}, {'d': {'id': 'ulcerated tumor in the body of the stomach'}}, {'d': {'id': 'H. pylori infection'}}, {'d': {'id': 'elevated lipids'}}, {'d': {'id': 'anxiety'}}, {'d': {'id': 'II diabetes mellitus'}}]

> Finished chain.


Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.FeatureDeprecationWarning} {category: DEPRECATION} {title: This feature is deprecated and will be removed in future versions.} {description: CALL subquery without a variable scope clause is now deprecated. Use CALL () { ... }} {position: line: 1, column: 1, offset: 0} for query: "CALL { CALL db.index.vector.queryNodes($index, $k, $embedding) YIELD node, score WITH collect({node:node, score:score}) AS nodes, max(score) AS max UNWIND nodes AS n RETURN n.node AS node, (n.score / max) AS score UNION CALL db.index.fulltext.queryNodes($keyword_index, $query, {limit: $k}) YIELD node, score WITH collect({node:node, score:score}) AS nodes, max(score) AS max UNWIND nodes AS n RETURN n.node AS node, (n.score / max) AS score } WITH node, max(score) AS score ORDER BY score DESC LIMIT $k RETURN reduce(str='', k IN ['text'] | str + '\\n' + k + ': ' + coalesce(node[k], '')) AS text, node {.*, `embedding

Graph data:
{'query': 'List the diseases', 'result': 'The diseases listed are moderately to poorly differentiated adenocarcinoma, type 2 diabetes, hypertension, stomach cancer, adenocarcinoma of the body of the stomach, ulcerated tumor in the body of the stomach, H. pylori infection, elevated lipids, anxiety, and II diabetes mellitus.'}
vector data:

text: examination of biopsy specimens of the stomach ulcer showed moderately to poorly differentiated adenocarcinoma. Staging computed tomography (CT) scans of the chest, abdomen and pelvis only showed prominent lymph nodes adjacent to the lesser curvature of the stomach within the gastrohepatic ligament and no evidence of metastatic disease.#Document 
text: II diabetes mellitus, hypertension, elevated lipids, gastroesophageal reflux disease (GERD), diverticulosis, endometriosis, kidney stones, anxiety and osteopenia. Her medications included atorvastatin, Inderal, lisinopril, metformin, omeprazole and Paxil. She had allergies to dyazide, 